## In order to start the breeding program, we have to have a few assumptions

Goal = 50,000 grams (size of a big dog)

Max Rats = 20 (total number of adult rats the lab support)

Initial Minimum Weight = 200 grams (Min weight of adult rat in initial population)

Initial Maximum Weight = 600 grams (Max weight of adult rat in initial population)

Initial Average Weight = 300 grams (Avg weight of adult rat in initial population)

Mutation Odds = 0.01 (Assumed probability of mutation)

Mutation Minimum = 0.5 (Scalar on rat weight of most weight reduction/least weight gain)

Mutation Maximum = 1.2 (Scalar on rat weight of most weight gain)

Litter Size = 8 (Number of pups per pair of mating rats)

Litter per Year = 10 (Number of litters per year per pair of mating rats)

Generation Limit = 500 (Max number of generation allowed for the breeding program)

In [1]:
import time 
import random
import statistics

# Assumed constants (weights are in gram)
GOAL = 50000
NUM_RATS = 20
INITIAL_MIN_WT = 200
INITIAL_MAX_WT = 600
INITIAL_AVG_WT = 300
MUTATE_ODDS = 0.01
MUTATE_MIN = 0.5
MUTATE_MAX = 1.2
LITTER_SIZE = 8
LITTERS_PER_YEAR = 10
GENERATION_LIMIT = 500

# Ensure even-number of rats for breeding pairs:
if NUM_RATS%2 != 0:
    NUM_RATS != 1

### Initialize the population

In [14]:
def populate_rat(num_rats,min_wt,max_wt,mode_wt):
    """Initialize the starting population of rats using triangle distribution"""
    return [int(random.triangular(min_wt,max_wt,mode_wt)) for i in range(num_rats)]

gen_0 = populate_rat(NUM_RATS,INITIAL_MIN_WT,INITIAL_MAX_WT,INITIAL_AVG_WT)
print(gen_0)
print(len(gen_0))

[519, 303, 336, 299, 395, 330, 466, 411, 293, 393, 264, 239, 440, 472, 320, 235, 334, 259, 414, 338]
20


### In order to represent my data, I will create a panda dataframe with min, max, and mean values that can later be graphed

In [12]:
import pandas as pd

df = pd.DataFrame(columns=['Min','Max','Mean','Raw'])

df.head()

,Min,Max,Mean,Raw


### Create selection defining equations

In [148]:
def fitness(population,goal):
    """
    Measure 'fitness' of population via mean / target
    The number will start low since the initial weight is low
    When fitness returns 1, the condition will have been met
    """
    ave = statistics.mean(population)
    return ave/goal

def selection(population,to_retain):
    """
    Cull population to retain only a specified number of rats
    Assumptions:
        _Male rats are always bigger than female rats
    To make it simply, I will simply split the ordered population by half
    Then, I will take the bigger half of each gender for the next round of selection
    """
    # step 1: sort population
    sorted_pop = sorted(population)

    # step 2: split population into female and male
    females = sorted_pop[:to_retain]
    males   = sorted_pop[to_retain:]

    # step 3: pick the upper part of each gender
    x = int(to_retain/2)
    females_retained = females[-x:]
    males_retained = males[-x:]

    return males_retained,females_retained

In [34]:
selection(gen_0,10)

tuple

### Create a breeding equation

In [32]:
def breed(males,females,litter_size):
    """
    This function will breed offsprings whose weight is randomly chosen between
    the min(mom's weight) and the max(dad's weight)...don't worry, we will add
    mutation later so that it is possible to outweight the dad

    In another word, this is offspring creation with no mutation
    """
    children = []
    random.shuffle(males) #shuffles the list to create random pairing
    random.shuffle(females)

    for male,female in zip(males,females): #zip will pair a male with a female
        for child in range(litter_size):
            child = random.randint(female,male)
            children.append(child)

    return children

In [43]:
selected = selection(gen_0,10)
len(breed(selected[1],selected[0],LITTER_SIZE))
#returned length makes sense because there are 5 pairs of mice, each will have 8 offsprings

40

### Create function for mutation of mice

In [161]:
def mutation(children,mutate_odds,mutate_min,mutate_max):
    """
    Function will randomly mutate rats based on predefined mutation odds
    """
    mutated = [x for x in children]
    for index,rat in enumerate(children):
        if True: # random.random() will generate a number between 0 and 1
            # if mutate odds is 0.01 --> then there is a 1% chance of mutation

            # if mutation occurs, the rat's weight is randomly adjusted in th range of our
            # mutation min and max
            mutated[index] = round(rat * random.uniform(mutate_min,mutate_max))
    
    return mutated

In [146]:
gen_next = breed(selected[1],selected[0],LITTER_SIZE)
mutated = mutation(gen_next,1,1,2)

data = {"gen":gen_next,"mutated":mutated}
compare_df = pd.DataFrame(data)
compare_df.head(5)
# as you can see, mutation is actually very rare, I took the liberty of adjusting mutate odd to 1 to show some results
# in the final function, instead of populating a new list, I will use the same variable to save some memory

,gen,mutated
0,426,652
1,451,675
2,352,608
3,395,474
4,445,599


In [147]:
def mutation(children,mutate_odds,mutate_min,mutate_max):
    """
    Function will randomly mutate rats based on predefined mutation odds
    """
    for index,rat in enumerate(children):
        if mutate_odds >= random.random(): # random.random() will generate a number between 0 and 1
            # if mutate odds is 0.01 --> then there is a 1% chance of mutation
            # if mutation occurs, the rat's weight is randomly adjusted in th range of our mutation min and max
            children[index] = round(rat * random.uniform(mutate_min,mutate_max))
    return children

### Create main() function

In [169]:
def main():
    """
    Bringing them all together
    """
    gen = 0
    
    parents = populate_rat(NUM_RATS,INITIAL_MIN_WT,INITIAL_MAX_WT,INITIAL_AVG_WT)

    fitness_coef = fitness(parents,GOAL)

    avg_wt = []
    min_wt = []
    max_wt = []
    fit_co = []

    while fitness_coef < 1 and gen < 1000000:
        selected_males,selected_females = selection(parents,NUM_RATS)
        children = breed(selected_males,selected_females,LITTER_SIZE)
        children = mutation(children,1,MUTATE_MIN,MUTATE_MAX)
        parents = selected_males + selected_females + children

        avg_wt.append(statistics.mean(parents))
        min_wt.append(max(parents))
        max_wt.append(max(parents))

        fitness_coef = fitness(parents,GOAL)
        fit_co.append(fitness_coef)
        
        gen+=1

    data = {
        'Min':min_wt,
        'Max':max_wt,
        'Mean':avg_wt,
        'Fitness':fit_co
    }
    df = pd.DataFrame(data)
    return df

In [170]:
df = main()

In [171]:
print(df)

        Min  Max   Mean   Fitness
0       528  528  455.4  0.009108
1       528  528  455.4  0.009108
2       528  528  455.4  0.009108
3       528  528  455.4  0.009108
4       528  528  455.4  0.009108
...     ...  ...    ...       ...
999995  528  528  455.4  0.009108
999996  528  528  455.4  0.009108
999997  528  528  455.4  0.009108
999998  528  528  455.4  0.009108
999999  528  528  455.4  0.009108

[1000000 rows x 4 columns]
